[Reference](https://medium.com/@sidathasiri/building-a-convolutional-neural-network-for-image-classification-with-tensorflow-f1f2f56bd83b)

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2021-07-14 11:26:01--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   157MB/s    in 0.4s    

2021-07-14 11:26:02 (157 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
!ls

sample_data


In [3]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

In [7]:
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_cats_dir = os.path.join(validation_dir, 'dogs')

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen = ImageDataGenerator( rescale = 1.0/255. )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        batch_size=20,
                                                        class_mode='binary',
                                                        target_size=(150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Constructing the model

In [10]:
import tensorflow as tf

In [12]:
tf.__version__

'2.5.0'

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Train the model

In [16]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['acc'])

In [17]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=15,
                              validation_steps=50,
                              verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
100/100 [==============================] - 71s 695ms/step - loss: 0.9084 - acc: 0.5785 - val_loss: 0.9427 - val_acc: 0.5010
Epoch 2/15
100/100 [==============================] - 69s 691ms/step - loss: 0.6479 - acc: 0.6500 - val_loss: 0.6358 - val_acc: 0.6710
Epoch 3/15
100/100 [==============================] - 69s 689ms/step - loss: 0.5613 - acc: 0.7210 - val_loss: 0.6459 - val_acc: 0.6520
Epoch 4/15
100/100 [==============================] - 69s 690ms/step - loss: 0.4849 - acc: 0.7575 - val_loss: 0.6201 - val_acc: 0.6840
Epoch 5/15
100/100 [==============================] - 69s 689ms/step - loss: 0.3835 - acc: 0.8235 - val_loss: 0.6626 - val_acc: 0.6870
Epoch 6/15
100/100 [==============================] - 69s 690ms/step - loss: 0.2930 - acc: 0.8750 - val_loss: 0.6389 - val_acc: 0.7190
Epoch 7/15
100/100 [==============================] - 69s 690ms/step - loss: 0.1967 - acc: 0.9200 - val_loss: 1.2167 - val_acc: 0.6540
Epoch 8/15
100/100 [==============================] - 6

# Image Augmentation

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [20]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [21]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        batch_size=20,
                                                        class_mode='binary',
                                                        target_size=(150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [22]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=15,
                              validation_steps=50,
                              verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
100/100 [==============================] - 78s 782ms/step - loss: 0.7011 - acc: 0.6560 - val_loss: 0.6287 - val_acc: 0.6410
Epoch 2/15
100/100 [==============================] - 78s 778ms/step - loss: 0.6351 - acc: 0.6580 - val_loss: 0.6414 - val_acc: 0.6650
Epoch 3/15
100/100 [==============================] - 78s 778ms/step - loss: 0.6183 - acc: 0.6865 - val_loss: 0.6024 - val_acc: 0.6580
Epoch 4/15
100/100 [==============================] - 78s 779ms/step - loss: 0.6154 - acc: 0.6660 - val_loss: 0.5837 - val_acc: 0.6800
Epoch 5/15
100/100 [==============================] - 78s 775ms/step - loss: 0.5985 - acc: 0.6850 - val_loss: 0.6067 - val_acc: 0.6910
Epoch 6/15
100/100 [==============================] - 78s 778ms/step - loss: 0.6056 - acc: 0.6815 - val_loss: 0.6762 - val_acc: 0.6410
Epoch 7/15
100/100 [==============================] - 77s 774ms/step - loss: 0.6087 - acc: 0.6935 - val_loss: 0.5711 - val_acc: 0.7050
Epoch 8/15
100/100 [==============================] - 7